# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [17]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [120]:
import findspark
findspark.init('C:\spark')
               
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
#df_visits = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [132]:
#write to parquet
#df_visits.write.parquet("sas_data")
df_visits=spark.read.parquet("data/immigration_data")
df_us_cities_demo = spark.read.csv("data/us-cities-demographics.csv", sep=";",  inferSchema=True, header=True)
df_airports = spark.read.csv("data/airport-codes_csv.csv", header=True)

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [122]:
# Performing cleaning tasks here
# df_visits.show(3)
# df_visits.describe()
df_visits.count()

3096313

In [133]:
# parse file containing valid country codes and create lookup table
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType
f = open("data/valid_country_codes.txt","r")
countries=[]
for line in f.readlines():
    line = line.replace("'", "").split('=')
    countries.append({"country_code":int(line[0].strip()), "country_name":line[1].strip()})

schema = StructType([
    StructField('country_code', IntegerType()),
    StructField('country_name', StringType())
])

df_countries = spark.createDataFrame(countries, schema)
df_countries.show(4)
df_countries.count()
df_countries.createOrReplaceTempView("lookup_country_codes")

+------------+--------------------+
|country_code|        country_name|
+------------+--------------------+
|         582|MEXICO Air Sea, a...|
|         236|         AFGHANISTAN|
|         101|             ALBANIA|
|         316|             ALGERIA|
+------------+--------------------+
only showing top 4 rows



In [134]:
# parse file containing valid airport codes and create lookup table
f = open("data/valid_airport_codes.txt", "r")
airports=[]
for line in f.readlines():
    line = line.replace("'", "").split('=')
    airports.append({"airport_code":line[0].strip(), "airport_name":line[1].strip()})

schema = StructType([
    StructField('airport_code', StringType()),
    StructField('airport_name', StringType())
])

df_airports = spark.createDataFrame(airports, schema)
df_airports.show(4)
df_airports.count()
df_airports.createOrReplaceTempView("lookup_airport_codes")


+------------+--------------------+
|airport_code|        airport_name|
+------------+--------------------+
|         ALC|           ALCAN, AK|
|         ANC|       ANCHORAGE, AK|
|         BAR|BAKER AAF - BAKER...|
|         DAC|   DALTONS CACHE, AK|
+------------+--------------------+
only showing top 4 rows



In [135]:
df_visits.createOrReplaceTempView("visits")

spark.sql("""select cicid,i94cit,i94res,i94port from visits 
where i94port not in (select airport_code from lookup_airport_codes)""").count()

spark.sql("""select cicid,i94cit,i94res,i94port from visits 
where i94cit not in (select country_code from lookup_country_codes)""").count()

386591

In [118]:
# rename columns(meaningful), fix column data types, convert arr/dep date to datetime
df_visits.createOrReplaceTempView("visits")

df_visits_clean = spark.sql("""SELECT cast(cicid as int) id, cast(i94cit as int) country_citizenship, 
cast(i94res as int) country_residence, i94port port, date_add('1960-01-01', cast(arrdate as int)) arrival_date, 
date_add('1960-01-01', cast(depdate as int)) departure_date, I94addr state, cast(i94bir as int) age,gender, 
cast(i94visa as int) visa_type,airline
FROM visits where i94port in (select airport_code from lookup_airport_codes)""")
df_visits_clean.show()


+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+
|     id|country_citizenship|country_residence|port|arrival_date|departure_date|state|age|gender|visa_type|airline|
+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+
|5761355|                297|              297| BGM|  2016-04-30|          null| null| 63|     F|        1|    348|
|5761356|                297|              297| BGM|  2016-04-30|    2016-08-08| null| 43|     M|        1|    348|
|5761357|                297|              297| BGM|  2016-04-30|    2016-08-19| null| 48|     M|        1|    348|
| 460085|                135|              135| BGM|  2016-04-03|    2016-04-06|   FL| 34|     M|        2|    R0E|
| 473180|                135|              135| BGM|  2016-04-03|    2016-08-21|   NY| 69|     M|        1|    R0E|
| 473264|                135|              135| BGM|  2016-04-03|       

In [119]:
df_visits.count()

3092098

In [13]:
df_us_cities_demo.count()
#df_us_cities_demo.show(4)
df_us_cities_demo.createOrReplaceTempView("demo")
spark.sql("SELECT * FROM demo where city='Quincy' and state='Massachusetts'").show()

+------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|  City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|  Hispanic or Latino| 2566|
|Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       

In [40]:
df_airports.count()
df_airports.createOrReplaceTempView("airports")
#spark.sql("select count(*) From airports where iso_country='US' and local_code != iata_code").show()
#spark.sql("""select name, iso_region, municipality, iata_code, local_code From airports 
#            where iso_country='US' and local_code != iata_code
#        """).show()
#spark.sql("select count(*) From airports where iso_country='US' and iata_code is null and local_code is not null").show()
spark.sql("select name, iso_region, municipality, iata_code, local_code From airports where iso_country='US' and iata_code is null and local_code is not null").show()

+--------------------+----------+------------+---------+----------+
|                name|iso_region|municipality|iata_code|local_code|
+--------------------+----------+------------+---------+----------+
|   Total Rf Heliport|     US-PA|    Bensalem|     null|       00A|
|Aero B Ranch Airport|     US-KS|       Leoti|     null|      00AA|
|        Lowell Field|     US-AK|Anchor Point|     null|      00AK|
|        Epps Airpark|     US-AL|     Harvest|     null|      00AL|
|      Fulton Airport|     US-OK|        Alex|     null|      00AS|
|      Cordes Airport|     US-AZ|      Cordes|     null|      00AZ|
|Goldstone /Gts/ A...|     US-CA|     Barstow|     null|      00CA|
| Williams Ag Airport|     US-CA|       Biggs|     null|      00CL|
|Kitchen Creek Hel...|     US-CA| Pine Valley|     null|      00CN|
| Grass Patch Airport|     US-FL|    Bushnell|     null|      00FA|
|  Ringhaver Heliport|     US-FL|   Riverview|     null|      00FD|
|   River Oak Airport|     US-FL|  Okeechobee|  

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

In [4]:
df_temperature.show(3)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 3 rows



In [5]:
df_temperature.createOrReplaceTempView("temperature")

In [16]:
#spark.sql("select * from temperature where dt> to_date('20000101','YYYYMMDD')").show(4)
spark.sql("select * from temperature where dt>= '2000-01-01' order by dt").show(4)
#spark.sql("select dt,count(*) from temperature group by dt order by dt desc").show(4)

+----------+-------------------+-----------------------------+-------+----------+--------+---------+
|        dt| AverageTemperature|AverageTemperatureUncertainty|   City|   Country|Latitude|Longitude|
+----------+-------------------+-----------------------------+-------+----------+--------+---------+
|2000-01-01|-16.703999999999994|          0.44799999999999995|Öskemen|Kazakhstan|  50.63N|   82.39E|
|2000-01-01|              3.065|          0.37200000000000005|Aalborg|   Denmark|  57.05N|   10.33E|
|2000-01-01|-15.574000000000002|                        0.664| Ürümqi|     China|  44.20N|   87.20E|
|2000-01-01|-3.1710000000000003|                        1.063|  Çorum|    Turkey|  40.99N|   34.08E|
+----------+-------------------+-----------------------------+-------+----------+--------+---------+
only showing top 4 rows



### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.